In [ ]:
# Installing kaggle to download the dataset.
!pip install kaggle

In [3]:
# We need kaggle credentials which needs to be stored in ~/.kaggle/kaggle.json
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

In [ ]:
import os
from getpass import getpass

# Reading in the contents of user's kaggle.json.
kaggle_secret = getpass('Enter the content of your kaggle.json: ')

# Saving into the previously created file.
with open('/root/.kaggle/kaggle.json', "w") as f:
  f.write(kaggle_secret)

In [ ]:
# Downloading and unzipping the training dataset.
!kaggle competitions download -c tpu-getting-started
!unzip tpu-getting-started.zip -d data
!rm tpu-getting-started.zip

In [6]:
import tensorflow as tf
import numpy as np
import re

# Paths to the training, validation and test datasets.
DATASET_PATH = "data"
PATH = DATASET_PATH + "/tfrecords-jpeg-512x512"
TRAINING_FILENAMES = tf.io.gfile.glob(PATH + "/train/*.tfrec")
VALIDATION_FILENAMES = tf.io.gfile.glob(PATH + "/val/*.tfrec")
TEST_FILENAMES = tf.io.gfile.glob(PATH + "/test/*.tfrec")

# Training related constants.
IMAGE_SIZE = [512, 512]
BATCH_SIZE = 32
NUM_OF_CLASSES = 104

In [7]:
# This function converts the raw image data to a [IMAGE_SIZE, 3] shaped array
# containing the normalized color intensity values for all channels.
def decode_image(image_data):
  # Extracting the image from the dataset.
  image = tf.image.decode_jpeg(image_data, channels=3)
  # Normalizing the color intensity values.
  image = (
      tf.cast(image, tf.float32) / 255.0
  )
  # Reshaping for 3 channels.
  image = tf.reshape(image, [*IMAGE_SIZE, 3])
  return image


# This function takes a raw labelled tfrecord and converts it to decoded image data
# and one hot encoded label.
def read_labelled_tfrecord(example):
  LABELLED_TFREC_FORMAT = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "class": tf.io.FixedLenFeature([], tf.int64),
  }
  # Converting the raw data to a python dictionary with LABELLED_TFREC_FORMAT.
  example = tf.io.parse_single_example(example, LABELLED_TFREC_FORMAT)
  # Decoding the image.
  image = decode_image(example["image"])
  # One-hot encode the label of the image.
  label = tf.cast(example["class"], tf.int32)
  one_hot = tf.one_hot(label, NUM_OF_CLASSES)
  return image, one_hot


# This function takes a raw unlabelled tfrecord and converts it to decoded image data
# and the id of the test image.
def read_unlabelled_tfrecord(example):
  UNLABELLED_TFREC_FORMAT = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "id": tf.io.FixedLenFeature([], tf.string),
  }
  # Converting raw data to a python dictionary with UNLABELLED_TFREC_FORMAT.
  example = tf.io.parse_single_example(example, UNLABELLED_TFREC_FORMAT)
  # Decoding the image.
  image = decode_image(example["image"])
  id_num = example["id"]
  return image, id_num


# This function takes several filenames (tfrecords) and creates a tensorflow
# dataset from them.
def load_dataset(filenames, labelled=True, ordered=False):
  # The order of the images doesn't matter so we are turning on ignore_order
  # this results in faster loading times.
  ignore_order = tf.data.Options()
  if not ordered:
    ignore_order.experimental_deterministic = False

  # Creating the dataset. We are using parallel read option because obviously
  # it results in faster loading times.
  dataset = tf.data.TFRecordDataset(
    filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE
  )
  dataset = dataset.with_options(ignore_order)
  # The final element of the pipeline is taking the raw tfrecord
  # and converting it to labelled on unlabelled input data with the upper functions.
  dataset = dataset.map(
    read_labelled_tfrecord if labelled else read_unlabelled_tfrecord,
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
  )
  return dataset

In [8]:
# This function creates the training dataset.
# repeat will be used, this will make sure that after reaching the end record
# we go back to the first.
# BATCH_SIZE will be used.
# We will prefetch 1 batch for accellerating the reading process.
def get_training_dataset():
  dataset = load_dataset(TRAINING_FILENAMES, labelled=True)
  dataset = dataset.repeat()
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(1)
  return dataset

# This function creates the validation dataset.
# BATCH_SIZE will be used.
# We will prefetch 1 batch for accellerating the reading process.
def get_validation_dataset(ordered=False):
  dataset = load_dataset(VALIDATION_FILENAMES, labelled=True, ordered=ordered)
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(1)
  return dataset


# This function creates the test dataset.
# BATCH_SIZE will be used.
# We will prefetch 1 batch for accellerating the reading process.
def get_test_dataset(ordered=False):
  dataset = load_dataset(TEST_FILENAMES, labelled=False, ordered=ordered)
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(1)
  return dataset


# This function counts the number of tfrecords in a dataset.
def count_data_items(filenames):
  n = [
    int(re.compile(r"-(\d*)\.").search(filename).group(1))
    for filename in filenames
  ]
  return np.sum(n)

In [9]:
# This will be used to calculate the number of batches needed to go through
# all the dataset in an epoch.
num_train_images = count_data_items(TRAINING_FILENAMES)

# Getting the train, validation and test datasets.
train_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()
test_dataset = get_test_dataset()

In [ ]:
# For hyperparameter-optimization keras-tuner will be used.
!pip install keras-tuner

In [ ]:
from tensorflow.keras import datasets, layers, models, callbacks, applications, optimizers
import keras_tuner as kt

def build_model(hp):
  # Using Transfer Learning technique for this project.
  # InceptionV3 pretrained model will be used.
  pre_trained_model = applications.InceptionV3(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
  # Freezing the weights of the model.
  for layer in pre_trained_model.layers:
    layer.trainable = False

  # Adding our own model on top of InceptionV3.
  x = pre_trained_model.output
  # Using pooling to decrease dimension.
  x = layers.GlobalAveragePooling2D()(x)
  # Putting some hidden layers depending on the current
  # hyperparameter-optimization iteration.
  for i in range(hp.Int('num_layers', min_value=1, max_value=3)):
    # A hidden layer with optimized size and activation function.
    x = layers.Dense(
        hp.Int('units_' + str(i), 256, 512, 32),
        activation='relu')(x)
    # Putting dropout after every hidden layer.
    x = layers.Dropout(hp.Choice('dropout_' + str(i), [0.3, 0.4, 0.5]))(x)
  # The output will be a probability distribution of classes.
  x = layers.Dense(NUM_OF_CLASSES, activation='softmax')(x)

  # Creating and compiling the final model from the pretrained model and our own model.
  model = models.Model(pre_trained_model.input, x)
  optimizer = optimizers.Adam(
      learning_rate=hp.Choice('learning_rate', [0.01, 0.001, 0.0001])
  )
  model.compile(
      optimizer=optimizer,
      loss='categorical_crossentropy',
      metrics=['acc']
  )
  return model

# Using keras-tuner for hyperparameter-optimization.
tuner = kt.Hyperband(
    build_model,
    objective='val_acc',
    max_epochs=5,
    factor=3,
    directory='best_model',
    project_name='FlowerClassificationModel'
)

# Using EarlyStopping to prevent overfitting.
es = callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')

# Searching with Hyperband in the hyperparameter-space.
tuner.search(train_dataset,
    validation_data=validation_dataset,
    epochs=25,
    steps_per_epoch=num_train_images / BATCH_SIZE,
    callbacks=[es])

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)

# Saving the best model so far.
cp = callbacks.ModelCheckpoint(filepath='model', monitor='val_acc', mode='max')
# Tensorboard for visualizing.
tb = callbacks.TensorBoard(update_freq=1)

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=50,
    steps_per_epoch=num_train_images / BATCH_SIZE,
    callbacks=[es, cp, tb])